In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 7 --executor-memory 4g --executor-cores 3 --driver-memory 3g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("lab5")
         .getOrCreate())

In [3]:
from pyspark.sql.functions import *

In [4]:
spark

In [5]:
!hdfs dfs -ls /labs/slaba05/

Found 2 items
-rw-r--r--   3 hdfs hdfs   24271415 2022-01-06 18:46 /labs/slaba05/lab05_test.csv
-rw-r--r--   3 hdfs hdfs  176160768 2022-01-06 18:46 /labs/slaba05/lab05_train.csv


In [6]:
train_df = spark.read.csv('/labs/slaba05/lab05_train.csv', header=True, sep=',', multiLine=True)
train_df.show(1, vertical=True)

-RECORD 0-------------------------------------------
 _c0                         | 333149               
 ID                          | 479990               
 CR_PROD_CNT_IL              | 0                    
 AMOUNT_RUB_CLO_PRC          | 0.0                  
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                  
 APP_REGISTR_RGN_CODE        | null                 
 PRC_ACCEPTS_A_POS           | 0.0                  
 PRC_ACCEPTS_A_TK            | 0.0                  
 TURNOVER_DYNAMIC_IL_1M      | 0.0                  
 CNT_TRAN_AUT_TENDENCY1M     | null                 
 SUM_TRAN_AUT_TENDENCY1M     | null                 
 AMOUNT_RUB_SUP_PRC          | 0.0                  
 PRC_ACCEPTS_A_AMOBILE       | 0.0                  
 SUM_TRAN_AUT_TENDENCY3M     | null                 
 CLNT_TRUST_RELATION         | null                 
 PRC_ACCEPTS_TK              | 0.0                  
 PRC_ACCEPTS_A_MTP           | 0.0                  
 REST_DYNAMIC_FDEP_1M        | 0.0            

In [7]:
train_df.count()

320764

In [8]:
train_df = train_df.na.drop("all").drop_duplicates()
train_df.count()

320764

In [9]:
train_df.select([count(when(col(c).isNull(), c)).alias(c) for c in train_df.columns]).show(vertical=True)

-RECORD 0-----------------------------
 _c0                         | 0      
 ID                          | 0      
 CR_PROD_CNT_IL              | 0      
 AMOUNT_RUB_CLO_PRC          | 34550  
 PRC_ACCEPTS_A_EMAIL_LINK    | 180814 
 APP_REGISTR_RGN_CODE        | 265914 
 PRC_ACCEPTS_A_POS           | 180814 
 PRC_ACCEPTS_A_TK            | 180814 
 TURNOVER_DYNAMIC_IL_1M      | 1      
 CNT_TRAN_AUT_TENDENCY1M     | 251229 
 SUM_TRAN_AUT_TENDENCY1M     | 251229 
 AMOUNT_RUB_SUP_PRC          | 34551  
 PRC_ACCEPTS_A_AMOBILE       | 180814 
 SUM_TRAN_AUT_TENDENCY3M     | 220560 
 CLNT_TRUST_RELATION         | 257829 
 PRC_ACCEPTS_TK              | 180814 
 PRC_ACCEPTS_A_MTP           | 180814 
 REST_DYNAMIC_FDEP_1M        | 1      
 CNT_TRAN_AUT_TENDENCY3M     | 220560 
 CNT_ACCEPTS_TK              | 180814 
 APP_MARITAL_STATUS          | 258989 
 REST_DYNAMIC_SAVE_3M        | 1      
 CR_PROD_CNT_VCU             | 1      
 REST_AVG_CUR                | 1      
 CNT_TRAN_MED_TENDENCY1M 

In [10]:
select_col = ["ID", "CR_PROD_CNT_IL", "AMOUNT_RUB_CLO_PRC", "TURNOVER_DYNAMIC_IL_1M", "AMOUNT_RUB_SUP_PRC", "REST_DYNAMIC_FDEP_1M",\
             "REST_DYNAMIC_SAVE_3M", "CR_PROD_CNT_VCU", "REST_AVG_CUR", "AMOUNT_RUB_NAS_PRC", "TRANS_COUNT_SUP_PRC",\
             "TRANS_COUNT_NAS_PRC", "CR_PROD_CNT_TOVR", "CR_PROD_CNT_PIL", "TURNOVER_CC", "TRANS_COUNT_ATM_PRC",\
             "AMOUNT_RUB_ATM_PRC", "TURNOVER_PAYM", "AGE", "CR_PROD_CNT_CC", "REST_DYNAMIC_FDEP_3M", "REST_DYNAMIC_IL_1M",\
             "CR_PROD_CNT_CCFP", "REST_DYNAMIC_CUR_1M", "REST_AVG_PAYM", "LDEAL_GRACE_DAYS_PCT_MED", "REST_DYNAMIC_CUR_3M",\
             "TURNOVER_DYNAMIC_CUR_1M", "REST_DYNAMIC_PAYM_3M", "REST_DYNAMIC_IL_3M", "TURNOVER_DYNAMIC_IL_3M", \
             "REST_DYNAMIC_PAYM_1M", "TURNOVER_DYNAMIC_CUR_3M", "PACK", "CLNT_SETUP_TENOR", "TURNOVER_DYNAMIC_PAYM_3M",\
             "TURNOVER_DYNAMIC_PAYM_1M", "TRANS_AMOUNT_TENDENCY3M", "REST_DYNAMIC_CC_1M", "TURNOVER_DYNAMIC_CC_1M",\
             "REST_DYNAMIC_CC_3M", "TURNOVER_DYNAMIC_CC_3M", "TARGET"]

train_df = train_df.select(select_col)
train_df.show(1, vertical=True)

-RECORD 0---------------------------------------
 ID                       | 285458              
 CR_PROD_CNT_IL           | 0                   
 AMOUNT_RUB_CLO_PRC       | 0.0                 
 TURNOVER_DYNAMIC_IL_1M   | 0.0                 
 AMOUNT_RUB_SUP_PRC       | 0.0                 
 REST_DYNAMIC_FDEP_1M     | 0.0                 
 REST_DYNAMIC_SAVE_3M     | 0.0                 
 CR_PROD_CNT_VCU          | 0                   
 REST_AVG_CUR             | 82438.1069          
 AMOUNT_RUB_NAS_PRC       | 0.0                 
 TRANS_COUNT_SUP_PRC      | 0.0                 
 TRANS_COUNT_NAS_PRC      | 0.0                 
 CR_PROD_CNT_TOVR         | 0                   
 CR_PROD_CNT_PIL          | 0                   
 TURNOVER_CC              | 0.0                 
 TRANS_COUNT_ATM_PRC      | 1.0                 
 AMOUNT_RUB_ATM_PRC       | 1.0                 
 TURNOVER_PAYM            | 0.0                 
 AGE                      | 528                 
 CR_PROD_CNT_CC     

In [11]:
test_df = spark.read.csv('/labs/slaba05/lab05_test.csv', header=True, sep=',', multiLine=True)
test_df = test_df.select(select_col[:-1])
test_df.count()

44399

In [12]:
test_df.select([count(when(col(c).isNull(), c)).alias(c) for c in test_df.columns]).show(vertical=True)

-RECORD 0------------------------
 ID                       | 0    
 CR_PROD_CNT_IL           | 0    
 AMOUNT_RUB_CLO_PRC       | 4794 
 TURNOVER_DYNAMIC_IL_1M   | 0    
 AMOUNT_RUB_SUP_PRC       | 4794 
 REST_DYNAMIC_FDEP_1M     | 0    
 REST_DYNAMIC_SAVE_3M     | 0    
 CR_PROD_CNT_VCU          | 0    
 REST_AVG_CUR             | 0    
 AMOUNT_RUB_NAS_PRC       | 4794 
 TRANS_COUNT_SUP_PRC      | 4794 
 TRANS_COUNT_NAS_PRC      | 4794 
 CR_PROD_CNT_TOVR         | 0    
 CR_PROD_CNT_PIL          | 0    
 TURNOVER_CC              | 0    
 TRANS_COUNT_ATM_PRC      | 4794 
 AMOUNT_RUB_ATM_PRC       | 4794 
 TURNOVER_PAYM            | 0    
 AGE                      | 0    
 CR_PROD_CNT_CC           | 0    
 REST_DYNAMIC_FDEP_3M     | 0    
 REST_DYNAMIC_IL_1M       | 0    
 CR_PROD_CNT_CCFP         | 0    
 REST_DYNAMIC_CUR_1M      | 0    
 REST_AVG_PAYM            | 0    
 LDEAL_GRACE_DAYS_PCT_MED | 0    
 REST_DYNAMIC_CUR_3M      | 0    
 TURNOVER_DYNAMIC_CUR_1M  | 0    
 REST_DYNAMIC_

In [13]:
train_df.dtypes

[('ID', 'string'),
 ('CR_PROD_CNT_IL', 'string'),
 ('AMOUNT_RUB_CLO_PRC', 'string'),
 ('TURNOVER_DYNAMIC_IL_1M', 'string'),
 ('AMOUNT_RUB_SUP_PRC', 'string'),
 ('REST_DYNAMIC_FDEP_1M', 'string'),
 ('REST_DYNAMIC_SAVE_3M', 'string'),
 ('CR_PROD_CNT_VCU', 'string'),
 ('REST_AVG_CUR', 'string'),
 ('AMOUNT_RUB_NAS_PRC', 'string'),
 ('TRANS_COUNT_SUP_PRC', 'string'),
 ('TRANS_COUNT_NAS_PRC', 'string'),
 ('CR_PROD_CNT_TOVR', 'string'),
 ('CR_PROD_CNT_PIL', 'string'),
 ('TURNOVER_CC', 'string'),
 ('TRANS_COUNT_ATM_PRC', 'string'),
 ('AMOUNT_RUB_ATM_PRC', 'string'),
 ('TURNOVER_PAYM', 'string'),
 ('AGE', 'string'),
 ('CR_PROD_CNT_CC', 'string'),
 ('REST_DYNAMIC_FDEP_3M', 'string'),
 ('REST_DYNAMIC_IL_1M', 'string'),
 ('CR_PROD_CNT_CCFP', 'string'),
 ('REST_DYNAMIC_CUR_1M', 'string'),
 ('REST_AVG_PAYM', 'string'),
 ('LDEAL_GRACE_DAYS_PCT_MED', 'string'),
 ('REST_DYNAMIC_CUR_3M', 'string'),
 ('TURNOVER_DYNAMIC_CUR_1M', 'string'),
 ('REST_DYNAMIC_PAYM_3M', 'string'),
 ('REST_DYNAMIC_IL_3M', 'stri

In [14]:
train_df = train_df.select([col(c).cast(FloatType()) if c != 'PACK' else col(c).cast(StringType()) for c in train_df.columns ])
test_df = test_df.select([col(c).cast(FloatType()) if c != 'PACK' else col(c).cast(StringType()) for c in test_df.columns ])
train_df.dtypes

[('ID', 'float'),
 ('CR_PROD_CNT_IL', 'float'),
 ('AMOUNT_RUB_CLO_PRC', 'float'),
 ('TURNOVER_DYNAMIC_IL_1M', 'float'),
 ('AMOUNT_RUB_SUP_PRC', 'float'),
 ('REST_DYNAMIC_FDEP_1M', 'float'),
 ('REST_DYNAMIC_SAVE_3M', 'float'),
 ('CR_PROD_CNT_VCU', 'float'),
 ('REST_AVG_CUR', 'float'),
 ('AMOUNT_RUB_NAS_PRC', 'float'),
 ('TRANS_COUNT_SUP_PRC', 'float'),
 ('TRANS_COUNT_NAS_PRC', 'float'),
 ('CR_PROD_CNT_TOVR', 'float'),
 ('CR_PROD_CNT_PIL', 'float'),
 ('TURNOVER_CC', 'float'),
 ('TRANS_COUNT_ATM_PRC', 'float'),
 ('AMOUNT_RUB_ATM_PRC', 'float'),
 ('TURNOVER_PAYM', 'float'),
 ('AGE', 'float'),
 ('CR_PROD_CNT_CC', 'float'),
 ('REST_DYNAMIC_FDEP_3M', 'float'),
 ('REST_DYNAMIC_IL_1M', 'float'),
 ('CR_PROD_CNT_CCFP', 'float'),
 ('REST_DYNAMIC_CUR_1M', 'float'),
 ('REST_AVG_PAYM', 'float'),
 ('LDEAL_GRACE_DAYS_PCT_MED', 'float'),
 ('REST_DYNAMIC_CUR_3M', 'float'),
 ('TURNOVER_DYNAMIC_CUR_1M', 'float'),
 ('REST_DYNAMIC_PAYM_3M', 'float'),
 ('REST_DYNAMIC_IL_3M', 'float'),
 ('TURNOVER_DYNAMIC_IL_3

In [15]:
test_df.dtypes

[('ID', 'float'),
 ('CR_PROD_CNT_IL', 'float'),
 ('AMOUNT_RUB_CLO_PRC', 'float'),
 ('TURNOVER_DYNAMIC_IL_1M', 'float'),
 ('AMOUNT_RUB_SUP_PRC', 'float'),
 ('REST_DYNAMIC_FDEP_1M', 'float'),
 ('REST_DYNAMIC_SAVE_3M', 'float'),
 ('CR_PROD_CNT_VCU', 'float'),
 ('REST_AVG_CUR', 'float'),
 ('AMOUNT_RUB_NAS_PRC', 'float'),
 ('TRANS_COUNT_SUP_PRC', 'float'),
 ('TRANS_COUNT_NAS_PRC', 'float'),
 ('CR_PROD_CNT_TOVR', 'float'),
 ('CR_PROD_CNT_PIL', 'float'),
 ('TURNOVER_CC', 'float'),
 ('TRANS_COUNT_ATM_PRC', 'float'),
 ('AMOUNT_RUB_ATM_PRC', 'float'),
 ('TURNOVER_PAYM', 'float'),
 ('AGE', 'float'),
 ('CR_PROD_CNT_CC', 'float'),
 ('REST_DYNAMIC_FDEP_3M', 'float'),
 ('REST_DYNAMIC_IL_1M', 'float'),
 ('CR_PROD_CNT_CCFP', 'float'),
 ('REST_DYNAMIC_CUR_1M', 'float'),
 ('REST_AVG_PAYM', 'float'),
 ('LDEAL_GRACE_DAYS_PCT_MED', 'float'),
 ('REST_DYNAMIC_CUR_3M', 'float'),
 ('TURNOVER_DYNAMIC_CUR_1M', 'float'),
 ('REST_DYNAMIC_PAYM_3M', 'float'),
 ('REST_DYNAMIC_IL_3M', 'float'),
 ('TURNOVER_DYNAMIC_IL_3

In [16]:
mode_list = ["CR_PROD_CNT_VCU", "CR_PROD_CNT_TOVR", "CR_PROD_CNT_PIL", "CR_PROD_CNT_CC", "CR_PROD_CNT_CCFP", "AGE", "PACK"]
mean_list = [i for i in select_col[1:-1] if i not in (mode_list + ["CR_PROD_CNT_IL"])]

In [17]:
for feature in mode_list:
    value = sorted(train_df.groupby(feature).count().collect(), key=lambda x: x[1], reverse=True)[0][0]
    train_df = train_df.na.fill(value=value, subset=[feature])
    test_df = test_df.na.fill(value=value, subset=[feature])

In [18]:
for feature in mean_list:
    value = train_df.select(F.mean(F.col(feature))).collect()[0][0]
    train_df = train_df.na.fill(value=value, subset=[feature])
    test_df = test_df.na.fill(value=value, subset=[feature])

In [19]:
train_df.select([count(when(col(c).isNull(), c)).alias(c) for c in train_df.columns]).show(vertical=True)

-RECORD 0-----------------------
 ID                       | 0   
 CR_PROD_CNT_IL           | 0   
 AMOUNT_RUB_CLO_PRC       | 0   
 TURNOVER_DYNAMIC_IL_1M   | 0   
 AMOUNT_RUB_SUP_PRC       | 0   
 REST_DYNAMIC_FDEP_1M     | 0   
 REST_DYNAMIC_SAVE_3M     | 0   
 CR_PROD_CNT_VCU          | 0   
 REST_AVG_CUR             | 0   
 AMOUNT_RUB_NAS_PRC       | 0   
 TRANS_COUNT_SUP_PRC      | 0   
 TRANS_COUNT_NAS_PRC      | 0   
 CR_PROD_CNT_TOVR         | 0   
 CR_PROD_CNT_PIL          | 0   
 TURNOVER_CC              | 0   
 TRANS_COUNT_ATM_PRC      | 0   
 AMOUNT_RUB_ATM_PRC       | 0   
 TURNOVER_PAYM            | 0   
 AGE                      | 0   
 CR_PROD_CNT_CC           | 0   
 REST_DYNAMIC_FDEP_3M     | 0   
 REST_DYNAMIC_IL_1M       | 0   
 CR_PROD_CNT_CCFP         | 0   
 REST_DYNAMIC_CUR_1M      | 0   
 REST_AVG_PAYM            | 0   
 LDEAL_GRACE_DAYS_PCT_MED | 0   
 REST_DYNAMIC_CUR_3M      | 0   
 TURNOVER_DYNAMIC_CUR_1M  | 0   
 REST_DYNAMIC_PAYM_3M     | 0   
 REST_DYNA

In [20]:
test_df.select([count(when(col(c).isNull(), c)).alias(c) for c in test_df.columns]).show(vertical=True)

-RECORD 0-----------------------
 ID                       | 0   
 CR_PROD_CNT_IL           | 0   
 AMOUNT_RUB_CLO_PRC       | 0   
 TURNOVER_DYNAMIC_IL_1M   | 0   
 AMOUNT_RUB_SUP_PRC       | 0   
 REST_DYNAMIC_FDEP_1M     | 0   
 REST_DYNAMIC_SAVE_3M     | 0   
 CR_PROD_CNT_VCU          | 0   
 REST_AVG_CUR             | 0   
 AMOUNT_RUB_NAS_PRC       | 0   
 TRANS_COUNT_SUP_PRC      | 0   
 TRANS_COUNT_NAS_PRC      | 0   
 CR_PROD_CNT_TOVR         | 0   
 CR_PROD_CNT_PIL          | 0   
 TURNOVER_CC              | 0   
 TRANS_COUNT_ATM_PRC      | 0   
 AMOUNT_RUB_ATM_PRC       | 0   
 TURNOVER_PAYM            | 0   
 AGE                      | 0   
 CR_PROD_CNT_CC           | 0   
 REST_DYNAMIC_FDEP_3M     | 0   
 REST_DYNAMIC_IL_1M       | 0   
 CR_PROD_CNT_CCFP         | 0   
 REST_DYNAMIC_CUR_1M      | 0   
 REST_AVG_PAYM            | 0   
 LDEAL_GRACE_DAYS_PCT_MED | 0   
 REST_DYNAMIC_CUR_3M      | 0   
 TURNOVER_DYNAMIC_CUR_1M  | 0   
 REST_DYNAMIC_PAYM_3M     | 0   
 REST_DYNA

In [21]:
from pyspark.ml.feature import CountVectorizer, RegexTokenizer
from pyspark.ml.linalg import Vectors, VectorUDT
convert_dense = F.udf(lambda vector: Vectors.dense(vector), VectorUDT())

In [22]:
tokenizer = RegexTokenizer(inputCol='PACK', outputCol='PACK_TOKENS', pattern=",")
cv = CountVectorizer(inputCol='PACK_TOKENS', outputCol="PACK_VECTOR")

In [23]:
train_df = tokenizer.transform(train_df)
cv_model = cv.fit(train_df)
train_df = cv_model.transform(train_df)

In [24]:
train_df.show(1, vertical=True)

-RECORD 0----------------------------------
 ID                       | 285458.0       
 CR_PROD_CNT_IL           | 0.0            
 AMOUNT_RUB_CLO_PRC       | 0.0            
 TURNOVER_DYNAMIC_IL_1M   | 0.0            
 AMOUNT_RUB_SUP_PRC       | 0.0            
 REST_DYNAMIC_FDEP_1M     | 0.0            
 REST_DYNAMIC_SAVE_3M     | 0.0            
 CR_PROD_CNT_VCU          | 0.0            
 REST_AVG_CUR             | 82438.11       
 AMOUNT_RUB_NAS_PRC       | 0.0            
 TRANS_COUNT_SUP_PRC      | 0.0            
 TRANS_COUNT_NAS_PRC      | 0.0            
 CR_PROD_CNT_TOVR         | 0.0            
 CR_PROD_CNT_PIL          | 0.0            
 TURNOVER_CC              | 0.0            
 TRANS_COUNT_ATM_PRC      | 1.0            
 AMOUNT_RUB_ATM_PRC       | 1.0            
 TURNOVER_PAYM            | 0.0            
 AGE                      | 528.0          
 CR_PROD_CNT_CC           | 0.0            
 REST_DYNAMIC_FDEP_3M     | 0.0            
 REST_DYNAMIC_IL_1M       | 0.0 

In [25]:
train_df = train_df.withColumn('PACK_VECTOR', convert_dense('PACK_VECTOR'))
train_df.show(1, vertical=True)

-RECORD 0----------------------------------------
 ID                       | 285458.0             
 CR_PROD_CNT_IL           | 0.0                  
 AMOUNT_RUB_CLO_PRC       | 0.0                  
 TURNOVER_DYNAMIC_IL_1M   | 0.0                  
 AMOUNT_RUB_SUP_PRC       | 0.0                  
 REST_DYNAMIC_FDEP_1M     | 0.0                  
 REST_DYNAMIC_SAVE_3M     | 0.0                  
 CR_PROD_CNT_VCU          | 0.0                  
 REST_AVG_CUR             | 82438.11             
 AMOUNT_RUB_NAS_PRC       | 0.0                  
 TRANS_COUNT_SUP_PRC      | 0.0                  
 TRANS_COUNT_NAS_PRC      | 0.0                  
 CR_PROD_CNT_TOVR         | 0.0                  
 CR_PROD_CNT_PIL          | 0.0                  
 TURNOVER_CC              | 0.0                  
 TRANS_COUNT_ATM_PRC      | 1.0                  
 AMOUNT_RUB_ATM_PRC       | 1.0                  
 TURNOVER_PAYM            | 0.0                  
 AGE                      | 528.0                


In [26]:
test_df = tokenizer.transform(test_df)
test_df = cv_model.transform(test_df)
test_df = test_df.withColumn('PACK_VECTOR', convert_dense('PACK_VECTOR'))
test_df.show(1, vertical=True)

-RECORD 0----------------------------------------
 ID                       | 519130.0             
 CR_PROD_CNT_IL           | 0.0                  
 AMOUNT_RUB_CLO_PRC       | 0.0                  
 TURNOVER_DYNAMIC_IL_1M   | 0.0                  
 AMOUNT_RUB_SUP_PRC       | 0.0                  
 REST_DYNAMIC_FDEP_1M     | 0.0                  
 REST_DYNAMIC_SAVE_3M     | 1.0                  
 CR_PROD_CNT_VCU          | 0.0                  
 REST_AVG_CUR             | 13916.819            
 AMOUNT_RUB_NAS_PRC       | 0.0                  
 TRANS_COUNT_SUP_PRC      | 0.0                  
 TRANS_COUNT_NAS_PRC      | 0.0                  
 CR_PROD_CNT_TOVR         | 0.0                  
 CR_PROD_CNT_PIL          | 0.0                  
 TURNOVER_CC              | 0.0                  
 TRANS_COUNT_ATM_PRC      | 1.0                  
 AMOUNT_RUB_ATM_PRC       | 1.0                  
 TURNOVER_PAYM            | 0.0                  
 AGE                      | 240.0                


In [27]:
from pyspark.ml.feature import VectorAssembler

In [28]:
vec_assembler = VectorAssembler(inputCols=(mode_list[:-1] + mean_list + ['PACK_VECTOR']), outputCol="features")

In [29]:
train_df = vec_assembler.transform(train_df)
train_df.show(1, vertical=True)

-RECORD 0----------------------------------------
 ID                       | 285458.0             
 CR_PROD_CNT_IL           | 0.0                  
 AMOUNT_RUB_CLO_PRC       | 0.0                  
 TURNOVER_DYNAMIC_IL_1M   | 0.0                  
 AMOUNT_RUB_SUP_PRC       | 0.0                  
 REST_DYNAMIC_FDEP_1M     | 0.0                  
 REST_DYNAMIC_SAVE_3M     | 0.0                  
 CR_PROD_CNT_VCU          | 0.0                  
 REST_AVG_CUR             | 82438.11             
 AMOUNT_RUB_NAS_PRC       | 0.0                  
 TRANS_COUNT_SUP_PRC      | 0.0                  
 TRANS_COUNT_NAS_PRC      | 0.0                  
 CR_PROD_CNT_TOVR         | 0.0                  
 CR_PROD_CNT_PIL          | 0.0                  
 TURNOVER_CC              | 0.0                  
 TRANS_COUNT_ATM_PRC      | 1.0                  
 AMOUNT_RUB_ATM_PRC       | 1.0                  
 TURNOVER_PAYM            | 0.0                  
 AGE                      | 528.0                


In [30]:
test_df = vec_assembler.transform(test_df)
test_df.show(1, vertical=True)

-RECORD 0----------------------------------------
 ID                       | 519130.0             
 CR_PROD_CNT_IL           | 0.0                  
 AMOUNT_RUB_CLO_PRC       | 0.0                  
 TURNOVER_DYNAMIC_IL_1M   | 0.0                  
 AMOUNT_RUB_SUP_PRC       | 0.0                  
 REST_DYNAMIC_FDEP_1M     | 0.0                  
 REST_DYNAMIC_SAVE_3M     | 1.0                  
 CR_PROD_CNT_VCU          | 0.0                  
 REST_AVG_CUR             | 13916.819            
 AMOUNT_RUB_NAS_PRC       | 0.0                  
 TRANS_COUNT_SUP_PRC      | 0.0                  
 TRANS_COUNT_NAS_PRC      | 0.0                  
 CR_PROD_CNT_TOVR         | 0.0                  
 CR_PROD_CNT_PIL          | 0.0                  
 TURNOVER_CC              | 0.0                  
 TRANS_COUNT_ATM_PRC      | 1.0                  
 AMOUNT_RUB_ATM_PRC       | 1.0                  
 TURNOVER_PAYM            | 0.0                  
 AGE                      | 240.0                


In [31]:
train_dataset = train_df.sampleBy('TARGET', fractions={0:0.8, 1:0.8}, seed=5757).cache()
valid_dataset = train_df.join(train_dataset, on="ID", how="leftanti").cache()

In [32]:
train_dataset.count()

256666

In [33]:
valid_dataset.count()

64098

In [34]:
valid_dataset = valid_dataset.na.drop("any")

In [35]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier

In [36]:
gbt = GBTClassifier(featuresCol='features', labelCol='TARGET', maxDepth=3, maxIter=25, stepSize=0.1)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="probability", labelCol="TARGET", metricName='areaUnderROC')

In [37]:
model_gbt = gbt.fit(train_dataset)

In [38]:
predictions_train = model_gbt.transform(train_dataset)
evaluator.evaluate(predictions_train)

0.805999423022986

In [39]:
predictions_valid = model_gbt.transform(valid_dataset)
evaluator.evaluate(predictions_valid)

0.8051405571718617

In [40]:
predictions_test = model_gbt.transform(test_df)
predictions_test = predictions_test.select("ID", "probability")
predictions_test.show(5)

+--------+--------------------+
|      ID|         probability|
+--------+--------------------+
|519130.0|[0.84736766117766...|
|234045.0|[0.95550500092477...|
|401256.0|[0.95661738805764...|
|551070.0|[0.93641680293479...|
|367285.0|[0.94864863509573...|
+--------+--------------------+
only showing top 5 rows



In [41]:
predictions_test = predictions_test.withColumn("ID", col("ID").cast(IntegerType()))
predictions_test.show(5)

+------+--------------------+
|    ID|         probability|
+------+--------------------+
|519130|[0.84736766117766...|
|234045|[0.95550500092477...|
|401256|[0.95661738805764...|
|551070|[0.93641680293479...|
|367285|[0.94864863509573...|
+------+--------------------+
only showing top 5 rows



In [42]:
udf_proba = F.udf(lambda x: float(x[1]), FloatType())

In [43]:
predictions_test = predictions_test.withColumn('target', udf_proba("probability"))
predictions_test.show(5)

+------+--------------------+-----------+
|    ID|         probability|     target|
+------+--------------------+-----------+
|519130|[0.84736766117766...| 0.15263234|
|234045|[0.95550500092477...|0.044494998|
|401256|[0.95661738805764...| 0.04338261|
|551070|[0.93641680293479...|0.063583195|
|367285|[0.94864863509573...|0.051351365|
+------+--------------------+-----------+
only showing top 5 rows



In [44]:
predictions_test = predictions_test.drop("probability")
predictions_test = predictions_test.withColumn("id", col("ID"))
predictions_test.show(5)

+------+-----------+
|    id|     target|
+------+-----------+
|519130| 0.15263234|
|234045|0.044494998|
|401256| 0.04338261|
|551070|0.063583195|
|367285|0.051351365|
+------+-----------+
only showing top 5 rows



In [45]:
my_file = predictions_test.toPandas()

In [46]:
my_file.to_csv('lab05.csv', sep='\t', index=False)